In [9]:
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from geopy import Nominatim
import folium
from branca.colormap import LinearColormap, StepColormap

%matplotlib inline


In [10]:
df = pd.read_csv('../data/round4/secondary_run1.csv')

In [12]:
df.head()

,sqft,gross_taxes_2017,area,sub_area,primary_broker,listing_id,property_type,age,school_info,price,address,type,link,features,strata_maintenance_fees,bath,strata_type,bed,amenities
0,630,914.0,Vancouver East,Mount Pleasant East,RE/MAX Select Realty,R2230676,Apt/Condo,Built in 1977 (41 yrs old),"[{'distance': '0.5 km', 'address': '2300 Guelp...",479900,204-2234 Prince Albert Street,Apt/Condo,https://www.rew.ca/properties/R2230676/204-223...,"Refrigerator, Stove",312.0,1,Freehold Strata,1,"Elevator, Shared Laundry"
1,1904,3991.0,Vancouver East,Hastings,Sutton Group West Coast Realty,R2231248,House,Built in 1910 (108 yrs old),"[{'distance': '0.3 km', 'address': '1950 Hasti...",1098800,1827 Pandora Street,House,https://www.rew.ca/properties/R2231248/1827-pa...,"Refrigerator, Storage Shed, Stove",NaN,2,Freehold NonStrata,4,NaN
2,841,1438.0,Vancouver East,Renfrew,RE/MAX Crest Realty,R2230613,Apt/Condo,Built in 2013 (5 yrs old),"[{'distance': '0.4 km', 'address': '2684 2nd A...",699000,2-2888 E 2nd Avenue,Apt/Condo,https://www.rew.ca/properties/R2230613/2-2888-...,ClthWsh/Dryr/Frdg/Stve/DW,262.0,2,Freehold Strata,2,"Elevator, In Suite Laundry"
3,3344,5016.0,Vancouver East,Fraserview East,RE/MAX Crest Realty,R2230712,House,Built in 2011 (7 yrs old),"[{'distance': '0.8 km', 'address': '2421 Scarb...",1988800,2526 SE Marine Drive,House,https://www.rew.ca/properties/R2230712/2526-se...,"Air Conditioning, ClthWsh/Dryr/Frdg/Stve/DW, C...",0.0,6,Freehold NonStrata,6,NaN
4,1210,4217.0,Vancouver East,Hastings East,Multiple Realty Ltd.,R2195938,House,Built in 1936 (82 yrs old),"[{'distance': '0.8 km', 'address': '250S Skeen...",1575000,3586 Trinity Street,House,https://www.rew.ca/properties/R2195938/3586-tr...,NaN,NaN,1,Freehold NonStrata,1,NaN


In [13]:
df.describe()

,sqft,gross_taxes_2017,price,strata_maintenance_fees,bath,bed
count,730.000000,661.000000,7.300000e+02,333.000000,730.000000,730.000000
mean,2204.509589,5926.803328,2.911873e+06,398.741742,3.071233,3.541096
std,1474.587746,6095.348213,3.323517e+06,422.484404,1.816060,2.005229
min,0.000000,0.000000,3.499000e+05,0.000000,0.000000,0.000000
25%,1098.500000,1910.000000,1.150000e+06,238.000000,2.000000,2.000000
50%,1930.500000,4693.000000,1.893500e+06,345.000000,3.000000,3.000000
75%,2963.500000,7924.000000,3.496750e+06,488.000000,4.000000,5.000000
max,12619.000000,58986.000000,3.800000e+07,4763.000000,8.000000,12.000000


## Let's use geocoding to get latlng coords of each house and add that to our DataFrame. We'll use these for visualizing on map later.

In [14]:
geocoder = Nominatim()
def address_to_latlng(address: str, num_retries: int = 3) -> tuple:
    """
    Convert an address to a point in LatLng with geocoding
    """
    print("search for {}".format(address))
    for i in range(num_retries):
        try:
            location = geocoder.geocode(query=address + ', Vancouver BC')
        except Exception as e:
            time.sleep(2)
            print("Error: {}. Retrying...({} retries left)".format(e, num_retries - i - 1))
            continue
        break
        
    if location is not None:
        print("found {}".format(location))
        return (location.latitude, location.longitude)
latlngs = []
for address in df['address']:
    latlng = address_to_latlng(address=address)
    latlngs.append(latlng)


search for 204-2234 Prince Albert Street
search for 1827 Pandora Street
found Pandora Street, Grandview-Woodland, Vancouver, Metro Vancouver, British Columbia, V5L, Canada
search for 2-2888 E 2nd Avenue
search for 2526 SE Marine Drive
found Southeast Marine Drive, Victoria-Fraserview, Vancouver, Metro Vancouver, British Columbia, V5P, Canada
search for 3586 Trinity Street
found Trinity Street, Vancouver, Metro Vancouver, British Columbia, V5K, Canada
search for 3536 William Street
found William Street, Grandview-Woodland, Vancouver, Metro Vancouver, British Columbia, V5L, Canada
search for 2735 Oxford Street
found Oxford Street, Burke Mountain, Oxford Heights, Coquitlam, Metro Vancouver, British Columbia, V3E 3A7, Canada
search for 2560 Dundas Street
found Dundas Street, Hastings-Sunrise, Vancouver, Metro Vancouver, British Columbia, V5L, Canada
search for 2867 Cambridge Street
found Cambridge Street, Capitol Hill, Burnaby, Metro Vancouver, British Columbia, V5B 1T7, Canada
search for 

found West 26th Avenue, Cambie Village, South Cambie, Vancouver, Metro Vancouver, British Columbia, V5Z, Canada
search for 3555 W 20th Avenue
found West 20th Avenue, South Granville, Arbutus Ridge, Vancouver, Metro Vancouver, British Columbia, V6L, Canada
search for 3309 W 19th Avenue
found West 19th Avenue, South Granville, Kitsilano, Vancouver, Metro Vancouver, British Columbia, V6L, Canada
search for 4047 W 18th Avenue
found West 18th Avenue, Arbutus Ridge, Vancouver, Metro Vancouver, British Columbia, V6L, Canada
search for 5503 Blenheim Street
found Blenheim Street, Arbutus Ridge, Vancouver, Metro Vancouver, British Columbia, V6N, Canada
search for 3805 W 38th Avenue
found West 38th Avenue, Cambie Village, South Cambie, Vancouver, Metro Vancouver, British Columbia, V5Z, Canada
search for 4208 Dunkirk Street
found Dunkirk Street, Dunbar, Vancouver, Metro Vancouver, British Columbia, V6R3S5, Canada
search for 3708 W 35th Avenue
found West 35th Avenue, Shaughnessy, Vancouver, Metro V

found West 2nd Avenue, Cambie Village, Fairview, Vancouver, Metro Vancouver, British Columbia, V5Z, Canada
search for 511-1633 Ontario Street
search for 1101-88 W 1st Avenue
search for 516-159 W 2nd Avenue
search for 311-388 W 1st Avenue
search for 504-89 W 2nd Avenue
search for 1402-445 W 2nd Avenue
search for 1401-38 W 1st Avenue
found West 1st Avenue, South Granville, Kitsilano, Vancouver, Metro Vancouver, British Columbia, V6J, Canada
search for 70-1425 Lamey's Mill Road
search for 211-1635 W 3rd Avenue
search for 621-159 W 2nd Avenue
search for 301-150 Athletes Way
search for 602-159 W 2nd Avenue
search for 309-181 W 1st Avenue
search for 305-1616 Columbia Street
search for 801-138 Athletes Way
search for 2405 Franklin Street
found Franklin Street, Hastings-Sunrise, Vancouver, Metro Vancouver, British Columbia, V5L, Canada
search for 207-1477 Fountain Way
search for 2761 Mcgill Street
found McGill Street, Hastings-Sunrise, Vancouver, Metro Vancouver, British Columbia, V5K, Canada


search for 1829 Stephens Street
found Stephens Street, Kitsilano, Vancouver, Metro Vancouver, British Columbia, V6K, Canada
search for 1956 W 15th Avenue
found West 15th Avenue, South Granville, Shaughnessy, Vancouver, Metro Vancouver, British Columbia, V6H, Canada
search for 3175 Point Grey Road
found Point Grey Road, Kitsilano, Vancouver, Metro Vancouver, British Columbia, V6K, Canada
search for 213-2450 Cornwall Avenue
search for 1838 W 12th Avenue
found West 12th Avenue, Cambie Village, Fairview, Vancouver, Metro Vancouver, British Columbia, V5Z, Canada
search for 715-2799 Yew Street
search for 301-1978 Vine Street
search for 301-1990 W 6th Avenue
search for 2765 W 8th Avenue
found West 8th Avenue, Point Grey, Vancouver, Metro Vancouver, British Columbia, V6R, Canada
search for 2515 W 8th Avenue
found West 8th Avenue, Point Grey, Vancouver, Metro Vancouver, British Columbia, V6R, Canada
search for 309-2565 Maple Street
search for 2511 W 8th Avenue
found West 8th Avenue, South Granv

search for 606-3533 Ross Drive
search for 568 E 59th Avenue
found East 59th Avenue, Victoria-Fraserview, Vancouver, Metro Vancouver, British Columbia, V5P, Canada
search for 401 E 55th Avenue
found East 55th Avenue, Victoria-Fraserview, Vancouver, Metro Vancouver, British Columbia, V5P, Canada
search for 421 W 59th Avenue
found West 59th Avenue, Marpole, Vancouver, Metro Vancouver, British Columbia, V6P, Canada
search for 2634 Napier Street
found Napier Street, Grandview-Woodland, Vancouver, Metro Vancouver, British Columbia, V5L, Canada
search for 217-2910 E Pender Street
search for 4922 Angus Drive
found Angus Drive, Marpole, Vancouver, Metro Vancouver, British Columbia, V6P, Canada
search for 1-36 W 13th Avenue
found West 13th Avenue, South Granville, Kitsilano, Vancouver, Metro Vancouver, British Columbia, V6K, Canada
search for 366 W 10th Avenue
found West 10th Avenue, South Granville, Fairview, Vancouver, Metro Vancouver, British Columbia, V6H, Canada
search for 2771 Manitoba Str

search for 220-2083 W 33rd Avenue
search for 1830 W 33rd Avenue
found West 33rd Avenue, Cambie Village, South Cambie, Vancouver, Metro Vancouver, British Columbia, V5Z, Canada
search for 2444 W King Edward Avenue
found West King Edward Avenue, Cambie Village, South Cambie, Vancouver, Metro Vancouver, British Columbia, V5Z, Canada
search for 407-4685 Valley Drive
search for 6069 Fremlin Street
found Fremlin Street, Oakridge, Vancouver, Metro Vancouver, British Columbia, V6P, Canada
search for 133 W 45th Avenue
found West 45th Avenue, Oakridge, Vancouver, Metro Vancouver, British Columbia, V6M, Canada
search for 805-6383 Cambie Street
search for 6174 Oak Street
found Oak Street, Cambie Village, South Cambie, Vancouver, Metro Vancouver, British Columbia, V5Z, Canada
search for 104 W 41st Avenue
found West 41st Avenue, Oakridge, Vancouver, Metro Vancouver, British Columbia, V5Z, Canada
search for 341 W 46th Avenue
found West 46th Avenue, Oakridge, Vancouver, Metro Vancouver, British Columb

found West 16th Avenue, South Granville, Shaughnessy, Vancouver, Metro Vancouver, British Columbia, V6J, Canada
search for 1203 W 38th Avenue
found West 38th Avenue, Cambie Village, South Cambie, Vancouver, Metro Vancouver, British Columbia, V5Z, Canada
search for 4511 Beverly Crescent
found Beverly Crescent, South Granville, Shaughnessy, Vancouver, Metro Vancouver, British Columbia, V6J, Canada
search for 3688 East Boulevard
found East Boulevard, South Granville, Kitsilano, Vancouver, Metro Vancouver, British Columbia, V6J, Canada
search for 5583 Laburnum Street
found Laburnum Street, Kerrisdale, Vancouver, Metro Vancouver, British Columbia, V6P, Canada
search for 602-4900 Cartier Street
search for 4788 Hudson Street
found Hudson Street, South Granville, Shaughnessy, Vancouver, Metro Vancouver, British Columbia, V6H, Canada
search for 4783 Marguerite Street
found Marguerite Street, South Granville, Shaughnessy, Vancouver, Metro Vancouver, British Columbia, V6M, Canada
search for 1390 

found East 57th Avenue, Victoria-Fraserview, Vancouver, Metro Vancouver, British Columbia, V5P, Canada
search for 574 E 51st Avenue
found East 51st Avenue, South Hill, Vancouver, Metro Vancouver, British Columbia, V5X, Canada
search for 732 E 57th Avenue
found East 57th Avenue, Victoria-Fraserview, Vancouver, Metro Vancouver, British Columbia, V5P, Canada
search for 215 E 64th Avenue
found East 64th Avenue, Sunset, Vancouver, Metro Vancouver, British Columbia, V5X, Canada
search for 759 E 63rd Avenue
found East 63rd Avenue, Sunset, Vancouver, Metro Vancouver, British Columbia, V5X, Canada
search for 28 E 60th Avenue
found East 60th Avenue, Sunset, Vancouver, Metro Vancouver, British Columbia, V5X, Canada
search for 1170 E 63rd Avenue
found East 63rd Avenue, Sunset, Vancouver, Metro Vancouver, British Columbia, V5X, Canada
search for 227 E 62nd Avenue
found East 62nd Avenue, Victoria-Fraserview, Vancouver, Metro Vancouver, British Columbia, V5P, Canada
search for 8054 Chester Street
fou

found Quesnel Drive, Arbutus Ridge, Vancouver, Metro Vancouver, British Columbia, V6L, Canada
search for 4319 Puget Drive
found Puget Drive, Arbutus Ridge, Vancouver, Metro Vancouver, British Columbia, V6L, Canada
search for 2713 W 23rd Avenue
found West 23rd Avenue, Arbutus Ridge, Vancouver, Metro Vancouver, British Columbia, V6L, Canada
search for 3906 Quesnel Drive
found Quesnel Drive, Arbutus Ridge, Vancouver, Metro Vancouver, British Columbia, V6L, Canada
search for 3788 Carnarvon Street
found Carnarvon Street, Dunbar, Vancouver, Metro Vancouver, British Columbia, V6L, Canada
search for 3833 Puget Drive
found Puget Drive, Arbutus Ridge, Vancouver, Metro Vancouver, British Columbia, V6L, Canada
search for 2266 W 21st Avenue
found West 21st Avenue, Dunbar, Vancouver, Metro Vancouver, British Columbia, V6L, Canada
search for 2922 W 21st Avenue
found West 21st Avenue, Dunbar, Vancouver, Metro Vancouver, British Columbia, V6L, Canada
search for 2758 W 22nd Avenue
found West 22nd Avenue

## Add the latlngs as a new column to the dataframe

In [15]:
df['latlng'] = latlngs
df.head()

,sqft,gross_taxes_2017,area,sub_area,primary_broker,listing_id,property_type,age,school_info,price,address,type,link,features,strata_maintenance_fees,bath,strata_type,bed,amenities,latlng
0,630,914.0,Vancouver East,Mount Pleasant East,RE/MAX Select Realty,R2230676,Apt/Condo,Built in 1977 (41 yrs old),"[{'distance': '0.5 km', 'address': '2300 Guelp...",479900,204-2234 Prince Albert Street,Apt/Condo,https://www.rew.ca/properties/R2230676/204-223...,"Refrigerator, Stove",312.0,1,Freehold Strata,1,"Elevator, Shared Laundry",None
1,1904,3991.0,Vancouver East,Hastings,Sutton Group West Coast Realty,R2231248,House,Built in 1910 (108 yrs old),"[{'distance': '0.3 km', 'address': '1950 Hasti...",1098800,1827 Pandora Street,House,https://www.rew.ca/properties/R2231248/1827-pa...,"Refrigerator, Storage Shed, Stove",NaN,2,Freehold NonStrata,4,NaN,"(49.2830253, -123.0637578)"
2,841,1438.0,Vancouver East,Renfrew,RE/MAX Crest Realty,R2230613,Apt/Condo,Built in 2013 (5 yrs old),"[{'distance': '0.4 km', 'address': '2684 2nd A...",699000,2-2888 E 2nd Avenue,Apt/Condo,https://www.rew.ca/properties/R2230613/2-2888-...,ClthWsh/Dryr/Frdg/Stve/DW,262.0,2,Freehold Strata,2,"Elevator, In Suite Laundry",None
3,3344,5016.0,Vancouver East,Fraserview East,RE/MAX Crest Realty,R2230712,House,Built in 2011 (7 yrs old),"[{'distance': '0.8 km', 'address': '2421 Scarb...",1988800,2526 SE Marine Drive,House,https://www.rew.ca/properties/R2230712/2526-se...,"Air Conditioning, ClthWsh/Dryr/Frdg/Stve/DW, C...",0.0,6,Freehold NonStrata,6,NaN,"(49.2109926, -123.0755534)"
4,1210,4217.0,Vancouver East,Hastings East,Multiple Realty Ltd.,R2195938,House,Built in 1936 (82 yrs old),"[{'distance': '0.8 km', 'address': '250S Skeen...",1575000,3586 Trinity Street,House,https://www.rew.ca/properties/R2195938/3586-tr...,NaN,NaN,1,Freehold NonStrata,1,NaN,"(49.2896967, -123.0256865)"


## Convert `age` into a numerical number

In [16]:
age_list = []
for age in df['age']:
    try:
        split = age.split('(')
        age = int(split[1].split(' ')[0])
    except Exception as e:
        age = None
    finally:
        age_list.append(age)
df['age'] = age_list

In [83]:
df.to_csv('../data/round4/secondary_run1_clean.csv', index=False)

## Combine 2 sets of data 

In [54]:
df2 = pd.DataFrame.from_csv('../data/round3/rew_round3_with_latlng.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [ ]:
# wierd...df2 has address as the index
df2['address'] = df2.index
df2.index = range(len(df2))

In [56]:
df_c = df.append(df2)

In [78]:
df_c = df.append(df2, ignore_index=True)

In [82]:
df_c = df_c.drop_duplicates(subset='address').reset_index()

In [85]:
df_c.to_csv('../data/all_jan12.csv', index=False)